# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 12 2023 11:16AM,254906,19,N17915,"NAPP Technologies, LLC",Released
1,Jan 12 2023 10:48AM,254905,19,60019240,"GUSA Granules USA, Inc.",Released
2,Jan 12 2023 10:40AM,254902,10,0086295881,ISDIN Corporation,Released
3,Jan 12 2023 10:40AM,254902,10,0086295898,ISDIN Corporation,Released
4,Jan 12 2023 10:40AM,254902,10,0086295922,ISDIN Corporation,Released
5,Jan 12 2023 10:40AM,254902,10,0086295933,ISDIN Corporation,Released
6,Jan 12 2023 10:40AM,254902,10,0086295935,ISDIN Corporation,Executing
7,Jan 12 2023 10:38AM,254901,10,MSP217466,"Methapharm, Inc.",Executing
8,Jan 12 2023 10:38AM,254901,10,MSP217479,"Methapharm, Inc.",Executing
9,Jan 12 2023 10:38AM,254901,10,MSP217464,Methapharm-G,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,254901,Released,2
33,254902,Executing,1
34,254902,Released,4
35,254905,Released,1
36,254906,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254899,NaN,NaN,1.0
254901,NaN,2.0,2.0
254902,NaN,1.0,4.0
254905,NaN,NaN,1.0
254906,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254770,7.0,3.0,2.0
254806,0.0,0.0,1.0
254810,0.0,1.0,0.0
254811,0.0,1.0,0.0
254813,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254770,7,3,2
254806,0,0,1
254810,0,1,0
254811,0,1,0
254813,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254770,7,3,2
1,254806,0,0,1
2,254810,0,1,0
3,254811,0,1,0
4,254813,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254770,7,3,2
1,254806,,,1
2,254810,,1,
3,254811,,1,
4,254813,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 12 2023 11:16AM,254906,19,"NAPP Technologies, LLC"
1,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc."
2,Jan 12 2023 10:40AM,254902,10,ISDIN Corporation
7,Jan 12 2023 10:38AM,254901,10,"Methapharm, Inc."
9,Jan 12 2023 10:38AM,254901,10,Methapharm-G
11,Jan 12 2023 10:20AM,254899,17,"Citieffe, Inc."
12,Jan 12 2023 9:44AM,254894,10,ISDIN Corporation
17,Jan 12 2023 9:44AM,254893,10,ISDIN Corporation
21,Jan 12 2023 8:57AM,254882,16,Sartorius Stedim Filters Inc.
22,Jan 12 2023 8:44AM,254878,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 12 2023 11:16AM,254906,19,"NAPP Technologies, LLC",,,1
1,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc.",,,1
2,Jan 12 2023 10:40AM,254902,10,ISDIN Corporation,,1,4
3,Jan 12 2023 10:38AM,254901,10,"Methapharm, Inc.",,2,2
4,Jan 12 2023 10:38AM,254901,10,Methapharm-G,,2,2
5,Jan 12 2023 10:20AM,254899,17,"Citieffe, Inc.",,,1
6,Jan 12 2023 9:44AM,254894,10,ISDIN Corporation,,,5
7,Jan 12 2023 9:44AM,254893,10,ISDIN Corporation,,,4
8,Jan 12 2023 8:57AM,254882,16,Sartorius Stedim Filters Inc.,,1,
9,Jan 12 2023 8:44AM,254878,10,ISDIN Corporation,,5,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 11:16AM,254906,19,"NAPP Technologies, LLC",1,,
1,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc.",1,,
2,Jan 12 2023 10:40AM,254902,10,ISDIN Corporation,4,1,
3,Jan 12 2023 10:38AM,254901,10,"Methapharm, Inc.",2,2,
4,Jan 12 2023 10:38AM,254901,10,Methapharm-G,2,2,
5,Jan 12 2023 10:20AM,254899,17,"Citieffe, Inc.",1,,
6,Jan 12 2023 9:44AM,254894,10,ISDIN Corporation,5,,
7,Jan 12 2023 9:44AM,254893,10,ISDIN Corporation,4,,
8,Jan 12 2023 8:57AM,254882,16,Sartorius Stedim Filters Inc.,,1,
9,Jan 12 2023 8:44AM,254878,10,ISDIN Corporation,,5,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 11:16AM,254906,19,"NAPP Technologies, LLC",1,,
1,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc.",1,,
2,Jan 12 2023 10:40AM,254902,10,ISDIN Corporation,4,1,
3,Jan 12 2023 10:38AM,254901,10,"Methapharm, Inc.",2,2,
4,Jan 12 2023 10:38AM,254901,10,Methapharm-G,2,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 11:16AM,254906,19,"NAPP Technologies, LLC",1.0,NaN,NaN
1,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
2,Jan 12 2023 10:40AM,254902,10,ISDIN Corporation,4.0,1.0,NaN
3,Jan 12 2023 10:38AM,254901,10,"Methapharm, Inc.",2.0,2.0,NaN
4,Jan 12 2023 10:38AM,254901,10,Methapharm-G,2.0,2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 11:16AM,254906,19,"NAPP Technologies, LLC",1.0,0.0,0.0
1,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
2,Jan 12 2023 10:40AM,254902,10,ISDIN Corporation,4.0,1.0,0.0
3,Jan 12 2023 10:38AM,254901,10,"Methapharm, Inc.",2.0,2.0,0.0
4,Jan 12 2023 10:38AM,254901,10,Methapharm-G,2.0,2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1784139,19.0,13.0,7.0
15,764545,55.0,60.0,33.0
16,509756,1.0,1.0,0.0
17,254899,1.0,0.0,0.0
19,1019542,23.0,42.0,0.0
21,3312743,10.0,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1784139,19.0,13.0,7.0
1,15,764545,55.0,60.0,33.0
2,16,509756,1.0,1.0,0.0
3,17,254899,1.0,0.0,0.0
4,19,1019542,23.0,42.0,0.0
5,21,3312743,10.0,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,19.0,13.0,7.0
1,15,55.0,60.0,33.0
2,16,1.0,1.0,0.0
3,17,1.0,0.0,0.0
4,19,23.0,42.0,0.0
5,21,10.0,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,19.0
1,15,Released,55.0
2,16,Released,1.0
3,17,Released,1.0
4,19,Released,23.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,17,19,21
Status,,,,,,
Completed,7.0,33.0,0.0,0.0,0.0,0.0
Executing,13.0,60.0,1.0,0.0,42.0,3.0
Released,19.0,55.0,1.0,1.0,23.0,10.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,17,19,21
0,Completed,7.0,33.0,0.0,0.0,0.0,0.0
1,Executing,13.0,60.0,1.0,0.0,42.0,3.0
2,Released,19.0,55.0,1.0,1.0,23.0,10.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,17,19,21
0,Completed,7.0,33.0,0.0,0.0,0.0,0.0
1,Executing,13.0,60.0,1.0,0.0,42.0,3.0
2,Released,19.0,55.0,1.0,1.0,23.0,10.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()